# The implementation of RAG chatbot with LangChain

In [1]:
import os
import openai

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv() 

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
llm_name = 'gpt-4o-mini'
data_path = '../data/faq_document.txt'

## Load data

In [3]:
# load the document and split it into chunks
loader = TextLoader(data_path)
documents = loader.load()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
len(docs)

12

In [4]:
persist_directory = 'docs/chroma/'
embedding_function = OpenAIEmbeddings()

## Create vector database and save it disk

In [5]:
# load it into Chroma and save to disk
db = Chroma.from_documents(docs, embedding_function, persist_directory=persist_directory)

print(f"Number of chuncks: {db._collection.count()}")

Number of chuncks: 24


## Check RAG chunks

In [6]:
# query it
query = 'Will my Medical Baseline Allowance transfer to my new address?'
docs = db.max_marginal_relevance_search(query, k=3, fetch_k=5)

# print results
for chunck in docs:
    print(chunck.page_content)
    print("#####################")

Will my Medical Baseline Allowance transfer to my new address?
Yes.  Your Medical Baseline Allowance should automatically transfer to your new address when you stop service at one location and start at another location. We always suggest to review your first full bill at your new home to ensure the allowance transferred properly.
#####################
You will still be required to meet minimum qualifications for the Medical Baseline Allowance at your new location.  If you have questions about those minimum qualifications, please visit our Medical Baseline Allowance page.
If you had the Medical Baseline Allowance at your previous location and the benefit did not transfer to your new location, please contact our Customer Service Department at 1-800-743-5000 and a Customer Service Representative will assist you.
#####################
Will my CARE discount transfer to my new address?
Yes.  Your CARE benefit should automatically transfer to your new address when you stop service at one loca

## Load vectordb from persist_directory saved above

In [7]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)
print(vectordb._collection.count())

24


## Initialize LLM

In [8]:
llm = ChatOpenAI(model_name=llm_name, temperature=0.2)

## Run without memory

In [15]:
# Build prompt
from langchain.prompts import PromptTemplate


template = """
You are a friendly and polite Pacific Gas and Electric Company FAQ Chatbot. 
Please answer user's questions truthfully and only based on the provided data, don't invent anything yourself. 
If you don't know the answer then kindly ask the user to call at 1-800-743-5000 and speak to a customer service representative.
You have access to FAQ page. Here is the data: {context}"
Question: {question}
Helpful Answer:
"""

In [16]:
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA

query = 'Are there yogurts?'
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_type="mmr"),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain.invoke({"query": query})
result["result"]

"I'm sorry, but I don't have information about yogurts. If you have questions related to PG&E services or programs, please feel free to ask! Otherwise, you can call 1-800-743-5000 to speak with a customer service representative for assistance."

## Run with memory

In [17]:
# add memory
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [18]:
from langchain.chains import ConversationalRetrievalChain


retriever=vectordb.as_retriever(search_type="mmr")
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
)

In [19]:
question = 'Will my Medical Baseline Allowance transfer to my new address?'
result = qa.invoke({"question": question})

print(result['answer'])

Yes, your Medical Baseline Allowance should automatically transfer to your new address when you stop service at one location and start at another location. It's recommended to review your first full bill at your new home to ensure the allowance transferred properly. However, you will still need to meet the minimum qualifications for the Medical Baseline Allowance at your new location.


In [20]:
question = "What are the minimum qualifications?"
result = qa.invoke({"question": question})

print(result['answer'])

I don't know.
